In [1]:
import boto3 
from boto3.dynamodb.conditions import Key, Attr 
client = boto3.client('dynamodb') 
dynamodb = boto3.resource('dynamodb') 
Table = dynamodb.Table('ads_passenger_processed')

In [2]:
response = Table.scan(FilterExpression=Attr('time').gt(0))

In [3]:
dict.keys(response) 

dict_keys(['Items', 'Count', 'ScannedCount', 'LastEvaluatedKey', 'ResponseMetadata'])

In [9]:
import pandas as pd 
df = pd.DataFrame.from_dict(pd.json_normalize(response['Items']), orient='columns')

In [11]:
from pandasgui import show 
show(df)

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
/tmp/ipykernel_32335/2052449813.py:2: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

/tmp/ipykernel_32335/2052449813.py:2: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

/tmp/ipykernel_32335/2052449813.py:2: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

/tmp/ipykernel_32335/2052449813.py:2: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future versio

In [7]:
type( response )

dict

In [8]:
type( Table )

boto3.resources.factory.dynamodb.Table